# Experimenting Prompt Engineering - Chatbot

In this notebook we will look at a few prompt engineering techniques. We will experiment by loading a 7 billion parameter Large Language Model (LLM) within the notebook environment itself and throwing some prompts its way to see what we can make it do.

After trying a few different prompts, we will run a simple chatbot using the prompt engineering techniques we explored. 

At the end, I will list some pointers in case you would like to build on this code, by dropping in other LLMs.

### Working Environment 

[![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/build-on-aws/generative-ai-prompt-engineering/blob/main/prompt-engineering-chatbot/prompt-engineering-chatbot.ipynb)


This notebook has been designed, written and tested to run for free on [Amazon SageMaker Studio Lab](https://studiolab.sagemaker.aws/) with CPU.  Studio Lab is a free machine learning (ML) development environment that provides compute and storage (up to 15GB) at no cost with NO credit card required.

You can sign up for Amazon SageMaker Studio Lab here: [https://studiolab.sagemaker.aws/]

> Whatever environment you end up using, make sure you have at least 12 GB of disk space available to run this code.

### Libraries
First, if needed, install `ctransformers` - a library based on `transformers` from [Hugging Face](https://huggingface.co/), a great open source set of libraries for working and experimenting with the underlying technology of generative AI.  

In [1]:
!pip install ctransformers>=0.2.24

In [10]:
!pip install langchain

     |████████████████████████████████| 2.0 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 55.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 39.5 MB/s eta 0:00:01��██████████████        | 798 kB 39.5 MB/s eta 0:00:01
     |████████████████████████████████| 18.2 MB 78.1 MB/s eta 0:00:01
     |████████████████████████████████| 44 kB 7.0 MB/s  eta 0:00:01
     |████████████████████████████████| 395 kB 77.6 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 100.2 MB/s eta 0:00:01
     |████████████████████████████████| 269 kB 84.8 MB/s eta 0:00:01
     |████████████████████████████████| 228 kB 96.6 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 8.6 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 610 kB 79.4 MB/s eta 0:00:01


In [2]:
from ctransformers import AutoModelForCausalLM

## Loading Mistral-7B-OpenOrca Model

The [Mistral-7B-OpenOrca](https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GGUF) model was fine-tuned on top of Mistral 7B using OpenOrca dataset. 

In [3]:
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GGUF", model_file="mistral-7b-openorca.Q4_K_M.gguf", model_type="mistral")


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

(…)a0bba72a0abe8aea7e127d7994cb/config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-openorca.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

# How do large language models work?

## Prompt engineering

LLMs are trained to predict next word, given a sequence of words. See [2nd diagram](https://huggingface.co/docs/transformers/main/llm_tutorial) on HF tutorial which demonstrates the concept. Let's try calling Mistral model and ask it to describe a concept:


In [5]:
print(llm("Explain photosythesis",max_new_tokens=250, temperature=0.1))



Photosynthesis is a process by which green plants and some other organisms use sunlight to synthesize or make their food (glucose) from carbon dioxide and water. This process occurs in the chloroplasts of plant cells, where light energy is converted into chemical energy.

The photosynthesis process can be divided into two main stages:

1. The Light-dependent reactions (Light phase): In this stage, the absorbed light energy is used to produce ATP (Adenosine Triphosphate) and NADPH (Nicotinamide adenine dinucleotide phosphate), which are high-energy molecules that can be used later in the Calvin cycle.

2. The Light-independent reactions (Calvin Cycle): In this stage, ATP and NADPH produced during the light-dependent reactions are used to convert carbon dioxide into glucose through a series of chemical reactions. This process does not require light energy but depends on the products generated in the previous stage.

Photosynthesis is essential for life on Earth because it provides oxyg

---
The model did pretty well, let's see if we can make the answer more concise:

In [6]:

print(llm("Explain photosythesis in three sentences.",max_new_tokens=150, temperature=0.1))



Photosynthesis is the process by which green plants, algae, and some bacteria convert sunlight into chemical energy in the form of glucose or other sugars. This process involves two main stages: the light-dependent reactions and the light-independent reactions (also known as the Calvin cycle).

In the first stage, light-dependent reactions, chlorophyll (a green pigment found in plant cells) absorbs sunlight and uses it to generate energy. This energy is used to produce ATP (adenosine triphosphate), which is a source of chemical energy for the cell, and NADPH (nicotinamide adenine dinucleotide ph


---
Better; however, the model didn't answer in three sentences. Let's try a different approach.

## Few-shot prompts
We can influence model's output style by using what's called few-shot prompting - a technique which shows the model the exact behavior we expect on few examples:

In [7]:
print(llm("""Explain precipitation in two sentences.
In meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls from clouds due to gravitational pull. The main forms of precipitation include drizzle, rain, sleet, snow, ice pellets, graupel and hail. 
--
Explain condensation in one sentence.
Condensation is the change of the state of matter from the gas phase into the liquid phase, and is the reverse of vaporization.
--
Explain photosynthesis in three sentences.""", max_new_tokens=150, temperature=0.1, stop="--"))


Photosynthesis is a process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll pigments. During this process, light energy is converted into chemical energy that can later be released to fuel the organisms' activities. In photosynthesis, carbon dioxide and water are absorbed from the atmosphere and a carbohydrate and oxygen are produced.



---
Great, it worked! You will notice we separated each example with double-dash sign and also signaled to LLM that it's a stop sequence, meaning, if it gets to the point where it wants to generate dash dash (because it's a pattern we showed it), it must stop.
# Instruction tuning

Building complex prompts to achieve simple objectives like this one can get complicated very fast, especially once we attempt more sophisticated tasks. Luckily, LLMs can be instruction tuned. This is done on special [datasets](https://huggingface.co/datasets/Open-Orca/OpenOrca) which fine-tune the model to follow directions as closely as possible. In other words, instead of providing examples in each prompt, this is done by changing the model weights as part of training processs. This Mistral model was actually instruction tuned and we just need to follow the format which was used during training:
```
<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
```
Let's try it:

In [8]:
print(llm("""<|im_start|>system
You are an AI assistant which gives helpful, detailed, and polite answers to the user's questions<|im_end|>
<|im_start|>user
Explain photosynthesis in just three sentences<|im_end|>
<|im_start|>assistant""",max_new_tokens=150, temperature=0.1))


 Photosynthesis is a process in which plants, algae, and some bacteria convert sunlight, water, and carbon dioxide into glucose (food) and oxygen. This process occurs in the chloroplasts of green plants and involves two stages: the light-dependent reactions and the light-independent reactions (Calvin cycle). Photosynthesis is essential for life on Earth as it provides energy, food, and oxygen to living organisms.


# Dyanamic prompting
Now that we know how to submit instructions to Mistral LLM, let's try assembling prompts dynamically - part of the prompt can be fixed, and part can be provided on the fly. We achieve this by creating prompt template with variables. The value for each variable is supplied in a separate statement and that statement can be executed further down in the code. Basically, this is a way to de-couple static part of the prompt from variable one, making the entire prompt dynamic. 

In [13]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """<|im_start|>system
You are an AI assistant which gives helpful, detailed, and polite answers to the user's questions<|im_end|>
<|im_start|>user
Given the content below, answer the question that follows. 
Content: {content}
Question: {question}<|im_end|>
<|im_start|>assistant
"""
)
prompt=prompt_template.format(content="ABC stands for Absolute Best Computer", question="What does ABC stand for?")
print(prompt)

<|im_start|>system
You are an AI assistant which gives helpful, detailed, and polite answers to the user's questions<|im_end|>
<|im_start|>user
Given the content below, answer the question that follows. 
Content: ABC stands for Absolute Best Computer
Question: What does ABC stand for?<|im_end|>
<|im_start|>assistant



In [14]:
llm(prompt,max_new_tokens=150, temperature=0.1)

' The acronym "ABC" in the given content stands for "Absolute Best Computer".'

In [15]:
# And what if we run it without any context:
llm("What does ABC stand for?",max_new_tokens=150, temperature=0.1)


'\n\nThe term "ABC" is an acronym that stands for "Alcohol, Beverages, and Cigarettes." It refers to the three primary categories of products that are commonly sold in convenience stores, supermarkets, and other retail establishments. These items are often displayed prominently at the front of the store or in a separate section, as they are high-margin products that generate significant revenue for the retailer.\n\nAlcohol: This category includes beer, wine, and spirits such as liquor and distilled beverages. Alcoholic beverages can be further divided into various types, including domestic and imported brands, as well as different styles of drinks like lagers, ales,'

---
In the example above you can see how without provided context, LLM uses its own "knowledge" to answer questions. The approach of providing context is used heavily in RAG (Retrieval Augmented Generation).

## More examples of LLM use cases:

For simpler questions/prompts we can often get away witout strict instruction formatting:

In [16]:
llm("""The following text came from OCR, correct obvious mistakes:
The students should use their notebots to practice installing required libraries and validating the code runs without any errands.
""",max_new_tokens=150, temperature=0.1)

'\nThe students should use their notebooks to practice installing required libraries and validating the code runs without any errors.'

In [17]:
llm("""Rewrite the following sentence in better English:
I think I want to apply for this position but don't know how, can you help?
""",max_new_tokens=150, temperature=0.1)

'\nThe correct version of the sentence is:\n"I believe I would like to apply for this position, but I am unsure about the process; could you offer some guidance?"'

In [18]:
llm("""I am getting the following error when attempting to run this pythong code, can you explain why?
  File "/home/studio-lab-user/sagemaker-studiolab-notebooks/ha.py", line 2, in <module>
    import zip
ModuleNotFoundError: No module named 'zip'
""",max_new_tokens=150, temperature=0.1)

'\nI think the error is because you are trying to import a module called "zip" which does not exist in Python. The "zip" function is a built-in function in Python and does not need to be imported as a module. If you want to use the zip function, you can simply call it directly without importing any module.\n\nFor example:\n```python\na = [1, 2, 3]\nb = [4, 5, 6]\nc = list(zip(a, b))\nprint(c)\n```\nThis code will output `[(1, 4), (2, 5), (3, 6)]`.'

In [19]:
llm("""Reduct PII from the following paragraph. Replace any PII with ###.
Paragraph:
Jeff Bezos lives at 1 Main St. Miami, FL 39812. His phone number is 111-123-4567.
""",max_new_tokens=150, temperature=0.1)

'\nReducted Paragraph:\nJeff Bezos lives at 1 Main St. Miami, FL ###. His phone number is 111-123-4567.'

---
The last one or two examples didn't quite work. Perhaps the model is not strong enough for this type of task. 

In [20]:
llm("""Create multiple choice question to test student's understanding of photosynthesis. The question must have at least three distractors. Indicate the correct answer.
""",max_new_tokens=250, temperature=0.1)

''

---
The above didn't work at all. Let's try with better prompt formatting:

In [21]:
print(llm("""<|im_start|>system
You are an AI assistant which creates assessments to help educators evaluate students knowledge<|im_end|>
<|im_start|>user
Create multiple choice question to test student's understanding of photosynthesis. The question must have at least three distractors. Indicate the correct answer.<|im_end|>
<|im_start|>assistant""",max_new_tokens=250, temperature=0.1))


 Question: In the process of photosynthesis, which molecule is converted into glucose?

A) Carbon dioxide (CO2)
B) Oxygen (O2)
C) Water (H2O)
D) Glucose (C6H12O6)

Correct Answer: D) Glucose (C6H12O6)


# Hallucinations

In [22]:
print(llm("""<|im_start|>system
You are an AI assistant which answers user questions politely and in great details<|im_end|>
<|im_start|>user
What type of new chemical solution AWS announced at reInvent this year?<|im_end|>
<|im_start|>assistant""",max_new_tokens=250, temperature=0.1))

:
 Amazon Web Services (AWS) has not specifically announced a new chemical solution at their re:Invent event. However, they have introduced various new services and features in different domains such as compute, storage, networking, databases, security, and more. Some of the notable announcements from the 2021 re:Invent event include:

1. Amazon SageMaker Studio Notebooks: A fully managed development environment for data scientists to build, train, and deploy machine learning models.

2. AWS Outposts: An on-premises hardware solution that brings native AWS services, infrastructure, and operating models to customer datacenters or edge locations.

3. Amazon SageMaker Canary: A feature that allows data scientists to gradually roll out their machine learning models in production while monitoring the performance and impact on end-users.

4. AWS Security Hub: A centralized platform for managing and organizing security alerts from multiple AWS services, third-party security tools, and automat

In [ ]:
print(llm("""<|im_start|>system
You are an AI assistant which answers user questions politely and in great details. Do not make up facts. Say I don't know if you have no information about something.<|im_end|>
<|im_start|>user
What type of new chemical solution AWS announced at reInvent this year?<|im_end|>
<|im_start|>assistant""",max_new_tokens=250, temperature=0.1))

# Chatbot

Let's make a simple chatbot.  There is no special library to include and no setting to apply to the LLM, all we need is prompt engineering!

Let's use what we know of prompts with in context learning, to create a simple chatbot. 

In [31]:
print(llm("""<|im_start|>system
You are an AI assistant chat bot which answers user questions concisely and stops<|im_end|>
<|im_start|>user
Who is Jeff Bezos?<|im_end|>
<|im_start|>assistant""",max_new_tokens=150, temperature=0.1))


 Jeff Bezos is the founder, chairman, and former CEO of Amazon.com, an e-commerce giant and technology company. He is also the owner of The Washington Post and has invested in various other businesses and ventures. Born on January 12, 1964, he is one of the richest people in the world with a significant influence on global business and technology.
<|im_end|>


In [32]:
print(llm("""<|im_start|>system
You are an AI assistant chat bot which answers user questions concisely and stops<|im_end|>
<|im_start|>user
How old is he?<|im_end|>
<|im_start|>assistant""",max_new_tokens=150, temperature=0.1))

: I cannot determine someone's age without knowing their birthdate. Please provide the person's birthdate, or let me know if you would like to ask about a different topic.


In [34]:
print(llm("""<|im_start|>system
You are an AI assistant chat bot which answers user questions concisely and stops<|im_end|>
<|im_start|>user
Given the following context, answer the question below.
Context: Who is Jeff Bezos? Jeff Bezos is the founder, chairman, and former CEO of Amazon.com, an e-commerce giant and technology company. He is also the owner of The Washington Post and has invested in various other businesses and ventures. Born on January 12, 1964, he is one of the richest people in the world with a significant influence on global business and technology.
Question: How old is he?<|im_end|>
<|im_start|>assistant
""",max_new_tokens=150, temperature=0.1))

 Jeff Bezos was born on January 12, 1964. As of now, his age can be calculated by subtracting his birth year from the current year. Please note that this information may change as he ages.


## Want to use a different model with this notebook?

Hugging Face have many models that you can use and drop in to code like this. But you may need to make  modifications depending on the model you choose. 

